# 📊 日別売上進捗管理ツール（シンプル版）

## 概要
各媒体の生データから日別売上データを作成し、5日ごとの進捗を前月・前々月と比較できるツールです。

## 処理フロー
1. ①〜④: 各媒体の生データを日別に集計
2. ⑤: 全媒体データを統合
3. ⑥: 連携シートからドロップイン・設備予約を取得
4. ⑦: 5日ごと進捗を計算
5. ⑧: HTMLレポートを生成・ダウンロード

In [ ]:
#@title ①インスタベース利用データ集計プログラム（日別版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone
from IPython.display import display, HTML

def perform_instabase_aggregation(df):
    """インスタベースのデータフレームを受け取り、日別集計を実行"""
    target_sales_col = '予約金額 (税込)'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    df = df.dropna(subset=['集計日', '利用時間_td'])

    # 日別集計
    group_keys = ['集計日', '施設名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 昨日以前のデータのみ
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)
    summary['設備名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']
    summary['媒体名'] = 'インスタベース'

    return summary[['月次タグ', '日付', '施設名', '設備名', 'プラン名', '件数', '利用時間', '売上', '媒体名']]

# ファイルアップロード
print("【手順1】インスタベースの予約データCSVをアップロードしてください。")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_insta_raw = pd.read_csv(io.BytesIO(uploaded[filename]), encoding=enc)
            break
        except: continue

    df_insta_daily = perform_instabase_aggregation(df_insta_raw)
    print(f"\n✅ インスタベース日別集計完了: {len(df_insta_daily)}件")
    print(f"   期間: {df_insta_daily['日付'].min()} 〜 {df_insta_daily['日付'].max()}")
    print(f"   売上合計: ¥{df_insta_daily['売上'].sum():,}")
    
    # 日別集計結果を表示
    print("\n📋 【日別集計データ（先頭20件）】")
    display(df_insta_daily.head(20))
    
    # 日別売上サマリー
    daily_summary = df_insta_daily.groupby('日付').agg({'売上': 'sum', '件数': 'sum'}).reset_index()
    daily_summary = daily_summary.sort_values('日付', ascending=False)
    print(f"\n📊 【日別売上サマリー（直近10日）】")
    display(daily_summary.head(10))
else:
    df_insta_daily = pd.DataFrame()
    print("ファイルがアップロードされませんでした。")

In [ ]:
#@title ②スペースマーケット売上明細集計プログラム（日別版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone
from IPython.display import display, HTML

def perform_spacemarket_aggregation(df):
    """スペースマーケットのデータフレームを受け取り、日別集計を実行"""
    target_sales_col = '成約金額'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    df['集計日'] = pd.to_datetime(df['実施日'], errors='coerce', format='mixed')

    if '利用開始時間' in df.columns and '利用終了時間' in df.columns:
        df['利用時間_td'] = pd.to_datetime(df['利用終了時間'], format='mixed', errors='coerce') - pd.to_datetime(df['利用開始時間'], format='mixed', errors='coerce')
    else:
        df['利用時間_td'] = pd.Timedelta(0)

    df = df.dropna(subset=['集計日'])

    group_keys = ['集計日', '施設名', 'プラン名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)
    summary['設備名'] = summary['スペース名']
    summary['媒体名'] = 'スペースマーケット'

    return summary[['月次タグ', '日付', '施設名', '設備名', 'プラン名', '件数', '利用時間', '売上', '媒体名']]

# ファイルアップロード
print("【手順2】スペースマーケットの予約データCSVをアップロードしてください。")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_spacemarket_raw = pd.read_csv(io.BytesIO(uploaded[filename]), encoding=enc)
            break
        except: continue

    df_spacemarket_daily = perform_spacemarket_aggregation(df_spacemarket_raw)
    print(f"\n✅ スペースマーケット日別集計完了: {len(df_spacemarket_daily)}件")
    print(f"   期間: {df_spacemarket_daily['日付'].min()} 〜 {df_spacemarket_daily['日付'].max()}")
    print(f"   売上合計: ¥{df_spacemarket_daily['売上'].sum():,}")
    
    # 日別集計結果を表示
    print("\n📋 【日別集計データ（先頭20件）】")
    display(df_spacemarket_daily.head(20))
    
    # 日別売上サマリー
    daily_summary = df_spacemarket_daily.groupby('日付').agg({'売上': 'sum', '件数': 'sum'}).reset_index()
    daily_summary = daily_summary.sort_values('日付', ascending=False)
    print(f"\n📊 【日別売上サマリー（直近10日）】")
    display(daily_summary.head(10))
else:
    df_spacemarket_daily = pd.DataFrame()
    print("ファイルがアップロードされませんでした。")

In [ ]:
#@title ③スペイシー利用データ集計プログラム（日別版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone
from IPython.display import display, HTML

def perform_spacee_aggregation(df):
    """スペイシーのデータフレームを受け取り、日別集計を実行"""
    target_sales_col = '差引合計売上金額（税込）'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            if isinstance(x, (int, float)): return x
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    if '予約ステータス' in df.columns:
        df = df[df['予約ステータス'] == '予約完了'].copy()

    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    df = df.dropna(subset=['集計日', '利用時間_td'])

    group_keys = ['集計日', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)
    summary['施設名'] = summary['スペース名']
    summary['設備名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']
    summary['媒体名'] = 'スペイシー'

    return summary[['月次タグ', '日付', '施設名', '設備名', 'プラン名', '件数', '利用時間', '売上', '媒体名']]

# ファイルアップロード
print("【手順3】スペイシーの予約データCSVをアップロードしてください。")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_spacee_raw = pd.read_csv(io.BytesIO(uploaded[filename]), encoding=enc)
            break
        except: continue

    df_spacee_daily = perform_spacee_aggregation(df_spacee_raw)
    print(f"\n✅ スペイシー日別集計完了: {len(df_spacee_daily)}件")
    print(f"   期間: {df_spacee_daily['日付'].min()} 〜 {df_spacee_daily['日付'].max()}")
    print(f"   売上合計: ¥{df_spacee_daily['売上'].sum():,}")
    
    # 日別集計結果を表示
    print("\n📋 【日別集計データ（先頭20件）】")
    display(df_spacee_daily.head(20))
    
    # 日別売上サマリー
    daily_summary = df_spacee_daily.groupby('日付').agg({'売上': 'sum', '件数': 'sum'}).reset_index()
    daily_summary = daily_summary.sort_values('日付', ascending=False)
    print(f"\n📊 【日別売上サマリー（直近10日）】")
    display(daily_summary.head(10))
else:
    df_spacee_daily = pd.DataFrame()
    print("ファイルがアップロードされませんでした。")

In [ ]:
#@title ④いいアプリ利用データ集計プログラム（日別版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone
from IPython.display import display, HTML

def perform_iiapp_aggregation(df_main, df_ext):
    """いいアプリの通常分と延長分を合算し、日別集計する関数"""
    MIN_SALES_THRESHOLD = 1

    def clean_currency(x):
        if pd.isna(x): return 0
        if isinstance(x, (int, float)): return x
        s = re.sub(r'[^0-9.-]', '', str(x).replace('￥', '').replace('¥', '').replace(',', ''))
        return pd.to_numeric(s, errors='coerce')

    for df in [df_main, df_ext]:
        if '売上' in df.columns:
            df['売上'] = df['売上'].apply(clean_currency).fillna(0)

    # 通常データの処理
    df_main = df_main[df_main['売上'] >= MIN_SALES_THRESHOLD].copy()
    df_main['集計日'] = pd.to_datetime(df_main['売上確定日'], errors='coerce', format='mixed')

    def clean_time_str(s):
        if pd.isna(s): return s
        return str(s).replace(', ', ' ').strip()

    start_t = pd.to_datetime(df_main['開始'].apply(clean_time_str), errors='coerce', format='mixed')
    end_act = pd.to_datetime(df_main['実際の終了時間'].apply(clean_time_str), errors='coerce', format='mixed')
    end_sch = pd.to_datetime(df_main['終了'].apply(clean_time_str), errors='coerce', format='mixed')

    df_main['利用時間_td'] = end_act.fillna(end_sch) - start_t
    df_main = df_main.dropna(subset=['集計日', '利用時間_td'])

    # 延長データの処理
    df_ext = df_ext[df_ext['売上'] >= MIN_SALES_THRESHOLD].copy()
    df_ext['集計日'] = pd.to_datetime(df_ext['売上確定日'], errors='coerce', format='mixed')
    df_ext['延長時間_td'] = pd.to_timedelta(df_ext['延長した分数'], unit='m', errors='coerce')
    df_ext = df_ext.dropna(subset=['集計日', '延長時間_td'])

    # 日別集計
    group_keys = ['集計日', '施設名', 'プラン名', '設備名']

    summary_main = df_main.groupby(group_keys, dropna=False).agg(
        件数=('施設名', 'size'),
        通常時間_td=('利用時間_td', 'sum'),
        通常売上=('売上', 'sum')
    ).reset_index()

    summary_ext = df_ext.groupby(group_keys, dropna=False).agg(
        延長時間_td=('延長時間_td', 'sum'),
        延長売上=('売上', 'sum')
    ).reset_index()

    summary = pd.merge(summary_main, summary_ext, on=group_keys, how='outer').fillna(0)

    # 昨日以前のデータのみ
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    def timedelta_to_hhmm(td):
        if pd.isna(td) or isinstance(td, (int, float)): return '0:00'
        ts = td.total_seconds()
        if ts < 0: return '0:00'
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = (summary['通常時間_td'].apply(lambda x: x if isinstance(x, pd.Timedelta) else pd.Timedelta(0)) +
                         summary['延長時間_td'].apply(lambda x: x if isinstance(x, pd.Timedelta) else pd.Timedelta(0))).apply(timedelta_to_hhmm)
    summary['売上'] = (summary['通常売上'] + summary['延長売上']).round().astype(int)
    summary['媒体名'] = 'いいアプリ'

    return summary[['月次タグ', '日付', '施設名', '設備名', 'プラン名', '件数', '利用時間', '売上', '媒体名']]

# ファイルアップロード
print("【手順4-1】いいアプリの【通常】予約データCSVをアップロードしてください。")
uploaded_main = files.upload()

print("\n【手順4-2】いいアプリの【延長】予約データCSVをアップロードしてください。")
uploaded_ext = files.upload()

if uploaded_main and uploaded_ext:
    # 通常データ読み込み
    filename_main = list(uploaded_main.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_iiapp_main = pd.read_csv(io.BytesIO(uploaded_main[filename_main]), encoding=enc)
            break
        except: continue

    # 延長データ読み込み
    filename_ext = list(uploaded_ext.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_iiapp_ext = pd.read_csv(io.BytesIO(uploaded_ext[filename_ext]), encoding=enc)
            break
        except: continue

    df_iiapp_daily = perform_iiapp_aggregation(df_iiapp_main, df_iiapp_ext)
    print(f"\n✅ いいアプリ日別集計完了: {len(df_iiapp_daily)}件")
    print(f"   期間: {df_iiapp_daily['日付'].min()} 〜 {df_iiapp_daily['日付'].max()}")
    print(f"   売上合計: ¥{df_iiapp_daily['売上'].sum():,}")
    
    # 日別集計結果を表示
    print("\n📋 【日別集計データ（先頭20件）】")
    display(df_iiapp_daily.head(20))
    
    # 日別売上サマリー
    daily_summary = df_iiapp_daily.groupby('日付').agg({'売上': 'sum', '件数': 'sum'}).reset_index()
    daily_summary = daily_summary.sort_values('日付', ascending=False)
    print(f"\n📊 【日別売上サマリー（直近10日）】")
    display(daily_summary.head(10))
else:
    df_iiapp_daily = pd.DataFrame()
    print("ファイルがアップロードされませんでした。")

In [ ]:
#@title ⑤全媒体データ統合（日別版）

import pandas as pd
from google.colab import files
import io
import unicodedata
from IPython.display import display, HTML

print("【手順5】タグ設置リストをアップロードしてください。")
print("ファイル名: 会議室シート_改 - タグ設置リスト.csv")
uploaded_tags = files.upload()

if uploaded_tags:
    filename_tags = list(uploaded_tags.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_tags = pd.read_csv(io.BytesIO(uploaded_tags[filename_tags]), encoding=enc)
            break
        except: continue

    print(f"\n✅ タグ設置リスト読み込み完了: {len(df_tags)}件")

    # 店舗名マッピングを作成
    all_media_dfs = []

    # 各媒体のデータを統合
    media_dfs = {
        'インスタベース': df_insta_daily if 'df_insta_daily' in dir() and len(df_insta_daily) > 0 else None,
        'スペースマーケット': df_spacemarket_daily if 'df_spacemarket_daily' in dir() and len(df_spacemarket_daily) > 0 else None,
        'スペイシー': df_spacee_daily if 'df_spacee_daily' in dir() and len(df_spacee_daily) > 0 else None,
        'いいアプリ': df_iiapp_daily if 'df_iiapp_daily' in dir() and len(df_iiapp_daily) > 0 else None,
    }

    for media_name, df in media_dfs.items():
        if df is not None and len(df) > 0:
            # タグ設置リストから該当媒体のマッピングを取得
            mapping_df = df_tags[df_tags['媒体名'] == media_name]
            facility_map = dict(zip(mapping_df['媒体側施設名'], mapping_df['統一施設名']))
            equipment_map = dict(zip(mapping_df['媒体側設備名'], mapping_df['統一設備名']))

            df_copy = df.copy()
            df_copy['統一店名'] = df_copy['施設名'].map(facility_map).fillna(df_copy['施設名'])
            df_copy['統一設備名'] = df_copy['設備名'].map(equipment_map).fillna(df_copy['設備名'])

            all_media_dfs.append(df_copy)
            print(f"  {media_name}: {len(df_copy)}件統合")

    if all_media_dfs:
        df_all_media_daily = pd.concat(all_media_dfs, ignore_index=True)
        # 不明データを除外
        df_all_media_daily = df_all_media_daily[
            (df_all_media_daily['統一店名'].notna()) &
            (df_all_media_daily['統一店名'] != '') &
            (df_all_media_daily['統一店名'] != '不明')
        ]
        print(f"\n✅ 全媒体統合完了: {len(df_all_media_daily)}件")
        print(f"\n【媒体別売上合計】")
        for media in df_all_media_daily['媒体名'].unique():
            total = df_all_media_daily[df_all_media_daily['媒体名'] == media]['売上'].sum()
            print(f"  {media}: ¥{total:,}")
        
        # 統合後のデータ表示
        print("\n📋 【統合後日別データ（先頭20件）】")
        display(df_all_media_daily.head(20))
        
        # 媒体×日付別サマリー
        print("\n📊 【日別×媒体別売上サマリー（直近5日）】")
        daily_media_summary = df_all_media_daily.groupby(['日付', '媒体名'])['売上'].sum().unstack(fill_value=0)
        daily_media_summary = daily_media_summary.sort_index(ascending=False)
        display(daily_media_summary.head(5))
    else:
        df_all_media_daily = pd.DataFrame()
        print("統合するデータがありません。")
else:
    df_all_media_daily = pd.DataFrame()
    print("タグ設置リストがアップロードされませんでした。")

In [ ]:
#@title ⑥連携シートからドロップイン・設備予約を取得（日別版）

import pandas as pd
import numpy as np
from google.colab import files
import io
import re
from IPython.display import display, HTML

print("【手順6】連携シートをアップロードしてください。")
print("ファイル名: 新_直営店舗売上データ - 連携.csv")
uploaded_renkei = files.upload()

def clean_num(series):
    return pd.to_numeric(series.astype(str).str.replace('円', '').str.replace(',', '').str.replace('¥', ''), errors='coerce').fillna(0)

if uploaded_renkei:
    filename_renkei = list(uploaded_renkei.keys())[0]
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try:
            df_renkei = pd.read_csv(io.BytesIO(uploaded_renkei[filename_renkei]), encoding=enc)
            break
        except: continue

    print(f"\n✅ 連携シート読み込み完了: {len(df_renkei)}件")
    print(f"   カラム: {list(df_renkei.columns)}")

    # 日付変換
    df_renkei['日付dt'] = pd.to_datetime(df_renkei['発生日'], errors='coerce')
    df_renkei = df_renkei.dropna(subset=['日付dt'])

    # 店舗名マッピング
    if 'df_tags' in dir():
        mapping_iiapp = df_tags[df_tags['媒体名'] == 'いいアプリ']
        store_map = dict(zip(mapping_iiapp['媒体側施設名'], mapping_iiapp['統一施設名']))
        df_renkei['統一店名'] = df_renkei['店名'].map(store_map).fillna(df_renkei['店名'])
    else:
        df_renkei['統一店名'] = df_renkei['店名']

    # 数値変換
    df_renkei['ドロップイン売上'] = clean_num(df_renkei['ドロップイン売上'])
    df_renkei['設備予約売上'] = clean_num(df_renkei['設備予約売上'])
    df_renkei['店舗プラン売上'] = clean_num(df_renkei['店舗プラン売上'])
    df_renkei['その他売上'] = clean_num(df_renkei['その他売上'])

    # 日別データとして整形
    daily_dropin_list = []
    daily_facility_list = []

    for idx, row in df_renkei.iterrows():
        store = row['統一店名']
        date_str = row['日付dt'].strftime('%Y-%m-%d')
        month_tag = row['日付dt'].strftime('%Y年%m月')

        if row['ドロップイン売上'] > 0:
            daily_dropin_list.append({
                '月次タグ': month_tag,
                '日付': date_str,
                '統一店名': store,
                'カテゴリ': 'ドロップイン',
                '売上': row['ドロップイン売上']
            })

        if row['設備予約売上'] > 0:
            daily_facility_list.append({
                '月次タグ': month_tag,
                '日付': date_str,
                '統一店名': store,
                'カテゴリ': '設備予約',
                '売上': row['設備予約売上']
            })

    df_dropin_daily = pd.DataFrame(daily_dropin_list)
    df_facility_daily = pd.DataFrame(daily_facility_list)

    print(f"\n【日別データ作成完了】")
    print(f"  ドロップイン: {len(df_dropin_daily)}件, 売上合計: ¥{df_dropin_daily['売上'].sum():,.0f}")
    print(f"  設備予約: {len(df_facility_daily)}件, 売上合計: ¥{df_facility_daily['売上'].sum():,.0f}")

    # ドロップイン日別データ表示
    if len(df_dropin_daily) > 0:
        print("\n📋 【ドロップイン日別データ（先頭15件）】")
        display(df_dropin_daily.head(15))
        
        # 日別サマリー
        dropin_daily_summary = df_dropin_daily.groupby('日付')['売上'].sum().reset_index()
        dropin_daily_summary = dropin_daily_summary.sort_values('日付', ascending=False)
        print("\n📊 【ドロップイン日別売上サマリー（直近10日）】")
        display(dropin_daily_summary.head(10))

    # 設備予約日別データ表示
    if len(df_facility_daily) > 0:
        print("\n📋 【設備予約日別データ（先頭15件）】")
        display(df_facility_daily.head(15))
        
        # 日別サマリー
        facility_daily_summary = df_facility_daily.groupby('日付')['売上'].sum().reset_index()
        facility_daily_summary = facility_daily_summary.sort_values('日付', ascending=False)
        print("\n📊 【設備予約日別売上サマリー（直近10日）】")
        display(facility_daily_summary.head(10))

    # 月次集計用にも保持
    df_renkei_monthly = df_renkei.copy()
    df_renkei_monthly['dt'] = pd.to_datetime(df_renkei_monthly['月次'], errors='coerce')
else:
    df_dropin_daily = pd.DataFrame()
    df_facility_daily = pd.DataFrame()
    df_renkei_monthly = pd.DataFrame()
    print("連携シートがアップロードされませんでした。")

In [ ]:
#@title ⑦5日ごと進捗データの計算

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, HTML

print("--- 5日ごと進捗データを計算中 ---\n")

# 全日別データを統合
all_daily_list = []

# 外部媒体データ
if 'df_all_media_daily' in dir() and len(df_all_media_daily) > 0:
    for idx, row in df_all_media_daily.iterrows():
        all_daily_list.append({
            '日付': row['日付'],
            '店舗': row['統一店名'],
            'カテゴリ': row['媒体名'],
            '売上': row['売上']
        })

# ドロップイン
if 'df_dropin_daily' in dir() and len(df_dropin_daily) > 0:
    for idx, row in df_dropin_daily.iterrows():
        all_daily_list.append({
            '日付': row['日付'],
            '店舗': row['統一店名'],
            'カテゴリ': 'ドロップイン',
            '売上': row['売上']
        })

# 設備予約
if 'df_facility_daily' in dir() and len(df_facility_daily) > 0:
    for idx, row in df_facility_daily.iterrows():
        all_daily_list.append({
            '日付': row['日付'],
            '店舗': row['統一店名'],
            'カテゴリ': '設備予約',
            '売上': row['売上']
        })

df_daily_all = pd.DataFrame(all_daily_list)
df_daily_all['日付dt'] = pd.to_datetime(df_daily_all['日付'])
df_daily_all['年月'] = df_daily_all['日付dt'].dt.to_period('M')
df_daily_all['日'] = df_daily_all['日付dt'].dt.day

# ★ 日別データの最新日付を自動取得
cutoff_dt = df_daily_all['日付dt'].max()
display_date = cutoff_dt.strftime('%Y年%m月%d日')
file_date_label = cutoff_dt.strftime('%Y%m%d')

print(f"✅ データ最新日（自動検出）: {display_date}")

# 当月・前月・前々月の設定
current_ym = cutoff_dt.to_period('M')
prev_ym = current_ym - 1
prev2_ym = current_ym - 2

print(f"\n【分析対象期間】")
print(f"  当月: {current_ym}")
print(f"  前月: {prev_ym}")
print(f"  前々月: {prev2_ym}")

print(f"\n✅ 全日別データ統合完了: {len(df_daily_all)}件")

# 統合データ確認
print("\n📋 【統合後の全日別データ（先頭20件）】")
display(df_daily_all.head(20))

print(f"\n【カテゴリ別売上合計】")
for cat in df_daily_all['カテゴリ'].unique():
    total = df_daily_all[df_daily_all['カテゴリ'] == cat]['売上'].sum()
    print(f"  {cat}: ¥{total:,.0f}")

# 日別×カテゴリ別クロス集計
print("\n📊 【日別×カテゴリ別売上クロス集計（直近10日）】")
daily_category_pivot = df_daily_all.groupby(['日付', 'カテゴリ'])['売上'].sum().unstack(fill_value=0)
daily_category_pivot = daily_category_pivot.sort_index(ascending=False)
display(daily_category_pivot.head(10))

# 5日ごとの進捗計算
milestone_days = [5, 10, 15, 20, 25]
current_day = cutoff_dt.day
active_milestones = [d for d in milestone_days if d <= current_day]
if current_day not in active_milestones:
    active_milestones.append(current_day)
active_milestones = sorted(active_milestones)

print(f"\n【計算対象マイルストーン】 {active_milestones}")

def get_cumulative_by_day(df, year_month, max_day):
    mask = (df['年月'] == year_month) & (df['日'] <= max_day)
    return df[mask].groupby(['店舗', 'カテゴリ'])['売上'].sum()

# 対象カテゴリ
target_categories = ['ドロップイン', '設備予約'] + list(df_daily_all['カテゴリ'].unique())
target_categories = list(set(target_categories))

progress_data = []
for milestone in active_milestones:
    current_cumsum = get_cumulative_by_day(df_daily_all, current_ym, milestone)
    prev_cumsum = get_cumulative_by_day(df_daily_all, prev_ym, milestone)
    prev2_cumsum = get_cumulative_by_day(df_daily_all, prev2_ym, milestone)

    all_keys = set(current_cumsum.index) | set(prev_cumsum.index) | set(prev2_cumsum.index)

    for (store, cat) in all_keys:
        current_val = current_cumsum.get((store, cat), 0)
        prev_val = prev_cumsum.get((store, cat), 0)
        prev2_val = prev2_cumsum.get((store, cat), 0)

        mom_ratio = (current_val / prev_val * 100) if prev_val > 0 else None
        mom2_ratio = (current_val / prev2_val * 100) if prev2_val > 0 else None

        progress_data.append({
            'マイルストーン': milestone,
            '基準日': f"{milestone}日時点",
            '店舗': store,
            'カテゴリ': cat,
            '当月累計': current_val,
            '前月同日累計': prev_val,
            '前月比': mom_ratio,
            '前々月同日累計': prev2_val,
            '前々月比': mom2_ratio,
        })

df_progress = pd.DataFrame(progress_data)
print(f"\n✅ 進捗データ作成完了: {len(df_progress)}件")

# 全店舗合計
df_progress_total = df_progress.groupby(['マイルストーン', '基準日', 'カテゴリ']).agg({
    '当月累計': 'sum',
    '前月同日累計': 'sum',
    '前々月同日累計': 'sum'
}).reset_index()

df_progress_total['前月比'] = (df_progress_total['当月累計'] / df_progress_total['前月同日累計'] * 100).replace([np.inf, -np.inf], np.nan)
df_progress_total['前々月比'] = (df_progress_total['当月累計'] / df_progress_total['前々月同日累計'] * 100).replace([np.inf, -np.inf], np.nan)
df_progress_total['店舗'] = '全店舗合計'

print("\n📊 【全店舗合計 - 5日ごと進捗サマリー】")
display(df_progress_total)

print("\n【全店舗合計 - 最新マイルストーン】")
latest_milestone = max(active_milestones)
latest_total = df_progress_total[df_progress_total['マイルストーン'] == latest_milestone]
for _, row in latest_total.iterrows():
    mom_str = f"{row['前月比']:.1f}%" if pd.notna(row['前月比']) else "-"
    print(f"  {row['カテゴリ']}: ¥{row['当月累計']:,.0f} (前月比: {mom_str})")

# 店舗別進捗も表示
print("\n📊 【店舗別進捗（最新マイルストーン）】")
latest_store_progress = df_progress[df_progress['マイルストーン'] == latest_milestone]
latest_store_progress = latest_store_progress.sort_values(['店舗', 'カテゴリ'])
display(latest_store_progress.head(30))

In [ ]:
#@title ⑧HTMLレポート生成 & ダウンロード

import pandas as pd
import numpy as np
import json
from google.colab import files

print("--- HTMLレポート生成中 ---")

# 月次データ用の13ヶ月カレンダー
months_13_dt = [(cutoff_dt.replace(day=1) - pd.DateOffset(months=i)) for i in range(12, -1, -1)]
month_labels = [d.strftime('%Y年%m月') for d in months_13_dt]

# 店舗リスト取得
all_stores = sorted(df_daily_all['店舗'].unique().tolist())
all_categories = sorted(df_daily_all['カテゴリ'].unique().tolist())

def calc_pct(curr, prev):
    if prev == 0: return "-" if curr == 0 else "新規"
    return f"{int((curr / prev) * 100)}%"

# 月次集計関数
def get_monthly_data(store_name=None):
    res_list = []
    for d, label in zip(months_13_dt, month_labels):
        row = {"month": label}
        ym = d.to_period('M')

        if store_name:
            df_filtered = df_daily_all[(df_daily_all['年月'] == ym) & (df_daily_all['店舗'] == store_name)]
        else:
            df_filtered = df_daily_all[df_daily_all['年月'] == ym]

        for cat in all_categories:
            row[cat] = int(df_filtered[df_filtered['カテゴリ'] == cat]['売上'].sum())

        # 変動売上（ドロップイン + 設備予約 + 外部媒体）
        row['変動売上'] = sum([row.get(c, 0) for c in all_categories if c not in ['店舗プラン', 'その他']])
        row['total_sales'] = sum([row.get(c, 0) for c in all_categories])

        res_list.append(row)

    # 前月比計算
    for i in range(len(res_list)):
        curr, prev = res_list[i], (res_list[i-1] if i > 0 else None)
        res_list[i]['total_sales_mom'] = calc_pct(curr.get('total_sales', 0), prev.get('total_sales', 0) if prev else 0)
        res_list[i]['変動売上_mom'] = calc_pct(curr.get('変動売上', 0), prev.get('変動売上', 0) if prev else 0)
        for cat in all_categories:
            res_list[i][f'{cat}_mom'] = calc_pct(curr.get(cat, 0), prev.get(cat, 0) if prev else 0)

    return res_list

# JSON構築
js_master = {
    "g_all": get_monthly_data(),
    "stores": [],
    "progress_total": df_progress_total.to_dict('records'),
    "progress_by_store": df_progress.to_dict('records'),
    "milestones": active_milestones,
    "categories": all_categories,
    "current_month": str(current_ym),
    "prev_month": str(prev_ym),
    "prev2_month": str(prev2_ym),
}

for store in all_stores:
    store_progress = df_progress[df_progress['店舗'] == store].to_dict('records')
    js_master["stores"].append({
        "id": f"st_{all_stores.index(store)}",
        "name": store,
        "summary": get_monthly_data(store),
        "progress": store_progress
    })

json_data_str = json.dumps(js_master, ensure_ascii=False, default=str)
categories_str = json.dumps(all_categories, ensure_ascii=False)

# HTML生成
html_template = f'''
<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1">
<title>日別売上進捗ダッシュボード - {display_date}</title>
<script src="https://cdn.plot.ly/plotly-2.24.1.min.js"></script>
<style>
:root {{ --primary: #22923f; --primary-light: #e8f8ed; --text: #2d3436; --bg: #f8f9fa; --border: #dfe6e9; --up: #e74c3c; --down: #2980b9; }}
body {{ font-family: sans-serif; margin: 0; display: flex; background: var(--bg); color: var(--text); }}
nav {{ width: 260px; background: #2f3640; height: 100vh; position: sticky; top: 0; overflow-y: auto; color: white; flex-shrink: 0; z-index: 1000; }}
nav h3 {{ padding: 20px; background: #222f3e; margin: 0; font-size: 1rem; }}
nav ul {{ list-style: none; padding: 0; margin: 0; }}
nav li a {{ display: block; padding: 12px 20px; color: #ced6e0; text-decoration: none; border-bottom: 1px solid #34495e; font-size: 0.82rem; }}
nav li a:hover {{ background: var(--primary); color: white; }}
.nav-section {{ padding: 8px 20px; background: #34495e; font-size: 0.75rem; color: #a4b0be; font-weight: bold; }}
.title-bar {{ position: sticky; top: 0; background: white; padding: 10px 25px; border-bottom: 3px solid var(--primary); box-shadow: 0 2px 5px rgba(0,0,0,0.08); z-index: 900; display: flex; justify-content: space-between; align-items: center; }}
.title-bar h2 {{ margin: 0; font-size: 1.2rem; color: var(--primary); }}
main {{ flex: 1; padding: 0; min-width: 0; }}
.section {{ padding: 10px 25px 80px; scroll-margin-top: 10px; }}
.card {{ background: white; border-radius: 12px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); margin-bottom: 35px; border: 1px solid var(--border); overflow: hidden; }}
.card-h {{ background: #f8f9fa; padding: 12px 20px; font-weight: bold; border-bottom: 1px solid var(--border); }}
.card-h.progress {{ background: #fff3cd; }}
.card-b {{ padding: 20px; }}
.chart-box {{ height: 380px; width: 100%; margin-bottom: 15px; }}
.table-c {{ overflow-x: auto; border-radius: 8px; border: 1px solid var(--border); margin-top: 15px; }}
table {{ width: 100%; border-collapse: collapse; font-size: 0.78rem; background: white; }}
th, td {{ border: 1px solid #eee; padding: 8px; text-align: right; }}
th {{ background: #f1f2f6; text-align: center; white-space: nowrap; }}
td:first-child {{ text-align: center; font-weight: bold; background: #fafafa; position: sticky; left: 0; }}
.mom {{ font-weight: bold; font-size: 0.75rem; }}
.mom.up {{ color: var(--up); }}
.mom.down {{ color: var(--down); }}
.tot {{ background: var(--primary-light); font-weight: bold; }}
.highlight {{ background: #fff3cd; }}
.menu-btn {{ display: none; position: fixed; bottom: 20px; right: 20px; background: var(--primary); color: white; border: none; border-radius: 50%; width: 60px; height: 60px; font-size: 24px; z-index: 2000; box-shadow: 0 5px 15px rgba(0,0,0,0.2); }}
@media (max-width: 1024px) {{ nav {{ position: fixed; left: -260px; transition: 0.3s; }} nav.open {{ left: 0; }} .menu-btn {{ display: block; }} }}
</style></head><body>
<button class="menu-btn" onclick="document.getElementById('sidebar').classList.toggle('open')">☰</button>
<nav id="sidebar"><h3>📊 売上進捗メニュー</h3><ul id="nav-ul"></ul></nav>
<main id="main-c"></main>
<script>
const data = {json_data_str};
const categories = {categories_str};
const content = document.getElementById('main-c');
const nav = document.getElementById('nav-ul');

let nHtml = '<div class="nav-section">📈 進捗分析</div>';
nHtml += '<li><a href="#progress-all">5日ごと進捗（全店舗）</a></li>';
nHtml += '<div class="nav-section">📊 月次レポート</div>';
nHtml += '<li><a href="#g-all">全店舗合計</a></li>';
nHtml += '<div class="nav-section">🏠 店舗別</div>';
data.stores.forEach(s => {{ nHtml += `<li><a href="#${{s.id}}">${{s.name}}</a></li>`; }});
nav.innerHTML = nHtml;

function getMomClass(m) {{ if(!m || m==="-" || m==="新規") return ""; const v = parseInt(m.replace('%','')); return v > 100 ? "up" : (v < 100 ? "down" : ""); }}
function fmtNum(n) {{ return n ? n.toLocaleString() : '0'; }}
function fmtPct(n) {{ return n ? n.toFixed(1) + '%' : '-'; }}

function getProgressTbl(progressData, milestones) {{
    if (!progressData || progressData.length === 0) return '<p>データなし</p>';
    let h = '<div class="table-c"><table><thead><tr><th>基準日</th><th>カテゴリ</th><th>当月累計</th><th>前月同日</th><th>前月比</th><th>前々月同日</th><th>前々月比</th></tr></thead><tbody>';
    const sortedMilestones = [...milestones].sort((a,b) => b - a);
    sortedMilestones.forEach(ms => {{
        const msData = progressData.filter(r => r['マイルストーン'] === ms);
        msData.forEach((r, idx) => {{
            const momClass = r['前月比'] ? (r['前月比'] > 100 ? 'up' : (r['前月比'] < 100 ? 'down' : '')) : '';
            const mom2Class = r['前々月比'] ? (r['前々月比'] > 100 ? 'up' : (r['前々月比'] < 100 ? 'down' : '')) : '';
            h += `<tr${{idx === 0 ? ' class="highlight"' : ''}}>`;
            h += `<td>${{r['基準日']}}</td><td>${{r['カテゴリ']}}</td>`;
            h += `<td class="tot">¥${{fmtNum(r['当月累計'])}}</td><td>¥${{fmtNum(r['前月同日累計'])}}</td>`;
            h += `<td class="mom ${{momClass}}">${{fmtPct(r['前月比'])}}</td>`;
            h += `<td>¥${{fmtNum(r['前々月同日累計'])}}</td><td class="mom ${{mom2Class}}">${{fmtPct(r['前々月比'])}}</td></tr>`;
        }});
    }});
    return h + '</tbody></table></div>';
}}

function getMonthlyTbl(rows, cats) {{
    let h = '<div class="table-c"><table><thead><tr><th rowspan="2">月次</th><th colspan="2" style="background:#e8f8ed">合計売上</th><th colspan="2" style="background:#fff3e0">変動売上</th>';
    cats.forEach(c => h += `<th colspan="2">${{c}}</th>`);
    h += '</tr><tr><th>金額</th><th>前月比</th><th>金額</th><th>前月比</th>';
    cats.forEach(c => h += '<th>金額</th><th>前月比</th>');
    h += '</tr></thead><tbody>';
    rows.forEach(r => {{
        h += `<tr><td>${{r.month}}</td>`;
        h += `<td class="tot">¥${{fmtNum(r.total_sales)}}</td><td class="mom ${{getMomClass(r.total_sales_mom)}}">${{r.total_sales_mom}}</td>`;
        h += `<td>¥${{fmtNum(r['変動売上'] || 0)}}</td><td class="mom ${{getMomClass(r['変動売上_mom'])}}">${{r['変動売上_mom'] || '-'}}</td>`;
        cats.forEach(c => {{
            h += `<td>¥${{fmtNum(r[c] || 0)}}</td><td class="mom ${{getMomClass(r[c+'_mom'])}}">${{r[c+'_mom'] || '-'}}</td>`;
        }});
        h += '</tr>';
    }});
    return h + '</tbody></table></div>';
}}

let sHtml = '';
sHtml += `<section id="progress-all" class="section">
    <div class="title-bar"><h2>📈 5日ごと売上進捗（全店舗合計）</h2><span>データ時点: {display_date}</span></div>
    <div class="card"><div class="card-h progress">⏱️ カテゴリ別進捗比較（前月比・前々月比）</div>
    <div class="card-b"><div id="c-progress" class="chart-box"></div><div id="t-progress"></div></div></div></section>`;

sHtml += `<section id="g-all" class="section"><div class="title-bar"><h2>📊 全店舗合計</h2><span>データ時点: {display_date}</span></div>
    <div class="card"><div class="card-h">月次売上内訳（カテゴリ別横並び＋前月比）</div>
    <div class="card-b"><div id="c-gall" class="chart-box"></div><div id="t-gall"></div></div></div></section>`;

data.stores.forEach(s => {{
    sHtml += `<section id="${{s.id}}" class="section"><div class="title-bar"><h2>🏠 ${{s.name}}</h2><span>データ時点: {display_date}</span></div>`;
    sHtml += `<div class="card"><div class="card-h progress">⏱️ 5日ごと売上進捗</div><div class="card-b"><div id="t-prog-${{s.id}}"></div></div></div>`;
    sHtml += `<div class="card"><div class="card-h">店舗全体実績</div><div class="card-b"><div id="c-s-${{s.id}}" class="chart-box"></div><div id="t-s-${{s.id}}"></div></div></div></section>`;
}});
content.innerHTML = sHtml;

const rendered = new Set();
function render(id) {{
    if(rendered.has(id)) return;
    const cfg = {{ responsive:true, displayModeBar:false }};
    const lay = {{ barmode:'stack', margin:{{t:40,b:40,l:60,r:15}}, font:{{size:11}}, legend:{{orientation:'h',y:-0.15}}, colorway:['#22923f','#3498db','#9b59b6','#f1c40f','#e67e22','#e74c3c','#1abc9c','#34495e'] }};

    if(id==='progress-all'){{
        const progData = data.progress_total;
        if(progData && progData.length > 0) {{
            const latestMs = Math.max(...data.milestones);
            const latestData = progData.filter(r => r['マイルストーン'] === latestMs);
            const trs = [{{
                x: latestData.map(r => r['カテゴリ']),
                y: latestData.map(r => r['前月比'] || 0),
                type: 'bar',
                name: '前月比',
                marker: {{ color: latestData.map(r => (r['前月比'] || 0) >= 100 ? '#e74c3c' : '#2980b9') }}
            }}];
            Plotly.newPlot('c-progress', trs, {{
                ...lay,
                title: `${{latestMs}}日時点 前月比`,
                shapes: [{{type:'line',x0:-0.5,x1:latestData.length-0.5,y0:100,y1:100,line:{{color:'red',width:2,dash:'dash'}}}}]
            }}, cfg);
        }}
        document.getElementById('t-progress').innerHTML = getProgressTbl(progData, data.milestones);
    }} else if(id==='g-all'){{
        const trs = categories.map(c=>({{x:data.g_all.map(r=>r.month),y:data.g_all.map(r=>r[c]),name:c,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
        Plotly.newPlot('c-gall',trs,lay,cfg);
        document.getElementById('t-gall').innerHTML = getMonthlyTbl(data.g_all, categories);
    }} else {{
        const s = data.stores.find(x => x.id === id); if(!s) return;
        if(s.progress && s.progress.length > 0) {{
            document.getElementById('t-prog-'+id).innerHTML = getProgressTbl(s.progress, data.milestones);
        }} else {{
            document.getElementById('t-prog-'+id).innerHTML = '<p>進捗データなし</p>';
        }}
        const trs = categories.map(c=>({{x:s.summary.map(r=>r.month),y:s.summary.map(r=>r[c]),name:c,type:'bar'}})).filter(t=>t.y.some(v=>v>0));
        Plotly.newPlot('c-s-'+id,trs,lay,cfg);
        document.getElementById('t-s-'+id).innerHTML = getMonthlyTbl(s.summary, categories);
    }}
    rendered.add(id);
}}

const obs = new IntersectionObserver((es)=>{{ es.forEach(e=>{{ if(e.isIntersecting) render(e.target.id); }}); }}, {{threshold:0.05}});
document.querySelectorAll('.section').forEach(s=>obs.observe(s));
</script></body></html>
'''

filename = f"日別売上進捗ダッシュボード_{file_date_label}.html"
with open(filename, "w", encoding="utf-8") as f:
    f.write(html_template)

print(f"\n✅ HTMLレポート生成完了: {filename}")
files.download(filename)
print("\n📥 ダウンロード開始...")

In [ ]:
#@title 【オプション】CSVでもダウンロード

from google.colab import files

print("CSVファイルを生成中...\n")

# 日別データ（全媒体統合）
daily_all_filename = f"日別データ_全媒体_{file_date_label}.csv"
df_daily_all.to_csv(daily_all_filename, index=False, encoding='utf-8-sig')

# 5日ごと進捗（全店舗）
progress_total_filename = f"5日ごと進捗_全店舗_{file_date_label}.csv"
df_progress_total.to_csv(progress_total_filename, index=False, encoding='utf-8-sig')

# 5日ごと進捗（店舗別）
progress_store_filename = f"5日ごと進捗_店舗別_{file_date_label}.csv"
df_progress.to_csv(progress_store_filename, index=False, encoding='utf-8-sig')

print(f"✅ 生成完了:")
print(f"   1. {daily_all_filename}")
print(f"   2. {progress_total_filename}")
print(f"   3. {progress_store_filename}")

files.download(daily_all_filename)
files.download(progress_total_filename)
files.download(progress_store_filename)

print("\n📥 ダウンロード完了")